In [10]:
# creating spectrograms from all the files, and saving split labelled versions to disk ready for machine learning
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
import cPickle as pickle
import numpy as np
import time
import random 

from scipy.ndimage.interpolation import zoom

#from data_helpers import load_annotations

import nolearn
import nolearn.lasagne
import lasagne.layers

from lasagne.layers import InputLayer, DimshuffleLayer
from lasagne.layers import DenseLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.nonlinearities import softmax, very_leaky_rectify as vlr
import theano

base = '/media/michael/Seagate/engage/alison_data/golden_set/'
annotation_pkl_dir = base + 'extracted/annotations/'
spec_pkl_dir = base + 'extracted/specs/'
log_dir = base + 'ml_runs/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
HWW = 15
SPEC_HEIGHT = 330
LEARN_LOG = True
DO_AUGMENTATION = False
DO_BATCH_NORM = True

In [12]:
# create a class for the spectrograms, from which we can get minbatches of data
class SpecSampler(object):
    
    def _ensure_correct_shape_spec(self, spec):
        if spec.ndim == 2:
            return spec[None, ...]
        else:
            return spec
    
    def __init__(self, specs, labels, hww):       
        self.specs = np.vstack(self._ensure_correct_shape_spec(spec) for spec in specs)
        self.median = np.median(self.specs, 2)
        self.labels = labels
        self.hww = hww
    
    def _ensure_shape(self, X):
        if X.shape[2] < self.hww * 2:
            return np.pad(X, ((0, 0), (0, 0), (0, self.hww * 2 - X.shape[2])), 'constant')
        else:
            return X
    
    def sample(self, num_per_class, seed=None):
        
        tic = time.time()
        num_samples = num_per_class * 2
        channels = self.specs.shape[0]
        height = self.specs.shape[1]
        
        if seed is not None:
            np.random.seed(seed)

        X = np.zeros((num_samples, channels, height, self.hww*2), np.float32)
        y = np.zeros(num_samples) * np.nan
        count = 0
        tic = time.time()
        
        for cls in [0, 1]:
            possible_locs = np.where(self.labels==cls)[0]

            if len(possible_locs) >= num_per_class:
                sampled_locs = np.random.choice(possible_locs, num_per_class, replace=False)

                for loc in sampled_locs:
                    X[count] = self._ensure_shape(
                        self.specs[:, :, (loc-self.hww):(loc+self.hww)])
                    y[count] = cls
                    count += 1

        # remove ones we couldn't get
        to_remove = np.isnan(y)
        return X[~to_remove], y[~to_remove].astype(np.int32)
        
        
#ss = SpecSampler([np.log((0.001+ melspec))[None, :100, :]], {0: bio_zoomed}, 10)

In [13]:
# load data and make list of specsamplers
samplers = []

for fname in os.listdir(spec_pkl_dir):
    
    # load spectrogram and annotations
    spec = pickle.load(open(spec_pkl_dir + fname))[:SPEC_HEIGHT, :]
    annots, wav, sample_rate = pickle.load(
        open(annotation_pkl_dir + fname))
        
    # reshape annotations
    for classname in annots:
        factor = float(spec.shape[1]) / annots[classname].shape[0]
        annots[classname] = zoom(annots[classname], factor)
        
    # create sampler
    if not LEARN_LOG:
        spec = np.log(0.001 + spec)
        spec = spec - np.median(spec, axis=1, keepdims=True)

    spec_stack = [spec]#, np.random.rand(*logspec.shape)]
    ss = SpecSampler(spec_stack, annots['anthrop'], HWW)
    ss.fname = fname
    samplers.append(ss)

In [14]:
class MyBatch(nolearn.lasagne.BatchIterator):
    def __iter__(self):
        bs = self.batch_size
        for sampler in self.X:
            xb, yb = sampler.sample(bs)
            
            num = xb.shape[0]
        
            # augmentation
            if DO_AUGMENTATION:
                xb *= (1.0 + np.random.randn(num, 1, 1, 1) * 0.1)
                xb += np.random.randn(num, 1, 1, 1) * 0.05

            if LEARN_LOG:
                # get medians
                meds = np.tile(sampler.median, (num, 1, 1))
                meds = np.tile(meds[:, :, :, None], (1, 1, 1, HWW*2))

                yield {'input': xb.astype(np.float32),
                      'input_med': meds.astype(np.float32)}, yb
            else:
                yield xb.astype(np.float32), yb

            
class MyBatchTest(nolearn.lasagne.BatchIterator):
    def __iter__(self):
        bs = self.batch_size
        for sampler in self.X:
            xb, yb = sampler.sample(bs, seed=10)

            if LEARN_LOG:
                meds = np.tile(sampler.median, (xb.shape[0], 1, 1))
                meds = np.tile(meds[:, :, :, None], (1, 1, 1, HWW*2))
                yield {'input': xb.astype(np.float32),
                      'input_med': meds.astype(np.float32)}, yb
            else:
                yield xb.astype(np.float32), yb
                

mb = MyBatch(20)
mb.X = samplers

for count, (xx, yy) in enumerate(mb):
    if LEARN_LOG:
        print xx['input'].max(), xx['input_med'].shape, yy.sum()
    else:
        print xx.shape, yy.sum(), xx.sum()
    break

89.8816 (20, 1, 330, 30) 20


In [15]:
import yaml             
# splits 
splits = yaml.load(open(base + 'splits/folds.yaml'))
train = splits[0] + splits[1]
test = splits[2]
                
class MyTrainSplit(nolearn.lasagne.TrainSplit):
    # custom data split
    def __call__(self, data, Yb, net):
        train_samplers = [xx for xx in samplers if xx.fname in train]
        test_samplers = [xx for xx in samplers if xx.fname in test]
        print len(train_samplers), len(test_samplers)
        return train_samplers, test_samplers, None, None

In [ ]:
# from lasagne.nonlinearities import elu as vlr
from lasagne.nonlinearities import softmax, very_leaky_rectify as vlr
from lasagne.layers import batch_norm, ElemwiseSumLayer, ExpressionLayer, DimshuffleLayer
from helpers import Log1Plus, ForgetSizeLayer
import theano.tensor as T

if not DO_BATCH_NORM:
    batch_norm = lambda x: x
net = {}

# main input layer, then logged
net['input'] = InputLayer((None, len(spec_stack), SPEC_HEIGHT, HWW*2), name='input')

if LEARN_LOG:
    off = lasagne.init.Constant(0.5)
    mult = lasagne.init.Constant(1.0)

    net['input_logged'] = Log1Plus(net['input'], off, mult)

    # logging the median and multiplying by -1
    net['input_med'] = InputLayer((None, len(spec_stack), SPEC_HEIGHT, HWW*2), name='input_med')
    net['med_logged'] = Log1Plus(net['input_med'], off=net['input_logged'].off, mult=net['input_logged'].mult)
    net['med_logged'] = ExpressionLayer(net['med_logged'], lambda X: -X)

    # summing the logged input with the negative logged median
    net['input'] = ElemwiseSumLayer((net['input_logged'], net['med_logged']))

net['conv1_1'] = batch_norm(
    ConvLayer(net['input'], 80, (spec.shape[0] - 5, 6), nonlinearity=vlr))
net['pool1'] = PoolLayer(net['conv1_1'], pool_size=(2, 2), stride=(2, 2), mode='max')
net['pool1'] = DropoutLayer(net['pool1'], p=0.5)
net['conv1_2'] = batch_norm(ConvLayer(net['pool1'], 80, (1, 3), nonlinearity=vlr))
# net['pool2'] = PoolLayer(net['conv1_2'], pool_size=(1, 2), stride=(1, 1))
net['pool2'] = DropoutLayer(net['conv1_2'], p=0.5)

net['fc6'] = batch_norm(DenseLayer(net['pool2'], num_units=512, nonlinearity=vlr))
net['fc6'] = DropoutLayer(net['fc6'], p=0.5)
net['fc7'] = batch_norm(DenseLayer(net['fc6'], num_units=512, nonlinearity=vlr))
net['fc7'] = DropoutLayer(net['fc7'], p=0.5)
net['fc8'] = DenseLayer(net['fc7'], num_units=2, nonlinearity=None)
net['prob'] = NonlinearityLayer(net['fc8'], softmax)

net = nolearn.lasagne.NeuralNet(
    layers=net['prob'],
    max_epochs=500,
    update=lasagne.updates.adam,
    update_learning_rate=0.0001,
#     update_momentum=0.975,
    verbose=1,
    batch_iterator_train=MyBatch(128),
    batch_iterator_test=MyBatchTest(128),
    train_split=MyTrainSplit(None),
    check_input=False
)

In [ ]:
net.fit(samplers, None)

26 14
# Neural Network with 1669540 learnable parameters

## Layer information

  #  name       size
---  ---------  --------
  0  input      1x330x30
  1             1x330x30
  2  input_med  1x330x30
  3             1x330x30
  4             1x330x30
  5             1x330x30
  6             80x6x25
  7             80x6x25
  8             80x6x25
  9             80x3x12
 10             80x3x12
 11             80x3x10
 12             80x3x10
 13             80x3x10
 14             80x3x10
 15             512
 16             512
 17             512
 18             512
 19             512
 20             512
 21             512
 22             512
 23             2
 24             2

  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  -----
      1       1.07726       0.84249      1.27866      0.42773  2.51s
      2       0.83687       0.79841      1.04816      0.45480  2.46s
      3       0.69005       0.75676  

In [9]:
print net.layers_['log1plus1'].off.get_value()
print net.layers_['log1plus3'].off.get_value()
print net.layers_['log1plus1'].mult.get_value()
print net.layers_['log1plus3'].mult.get_value()

KeyError: 'log1plus1'

In [ ]:
def plot_loss(net):
    train_loss = [row['train_loss'] for row in net.train_history_]
    valid_loss = [row['valid_loss'] for row in net.train_history_]
    plt.plot(train_loss, label='train loss')
    plt.plot(valid_loss, label='valid loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    return plt
plot_loss(net)

In [ ]:
y = [0.197185359589, 0.518698018591, 0.581381482387, 0.664551736791, 0.699983182485]
x = [100, 500, 1000, 2500, 5000]
